In [1]:
#Upload your kaggle API json file:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"vamsipvs","key":"11b9c77decceaea998a95ec44402006f"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d ashirwadsangwan/imdb-dataset

 99% 1.43G/1.44G [00:13<00:00, 112MB/s] 
100% 1.44G/1.44G [00:13<00:00, 115MB/s]


In [4]:
#Define the path and name of the directory to be created:
#This is the path where we will extract the dataset:
import os
path = "/content/imdb-dataset"

try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
   print ("Successfully created the directory %s " % path)

Successfully created the directory /content/imdb-dataset 


In [5]:
#Extracts the Images from the Zip file downloaded to the path we created above:
from zipfile import ZipFile
file_name = "imdb-dataset.zip"
with ZipFile(file_name, 'r') as zip:
  zip.extractall(path = "/content/imdb-dataset/")
  print('Done')

Done


In [6]:
!sudo apt-get update

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,418 kB]
Hit:12 http://ppa.launchpad.net/cran/libg

In [7]:
!sudo apt-get install

Reading package lists... Done
Building dependency tree       
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 81 not upgraded.


In [8]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark


In [9]:

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

import findspark
findspark.init("spark-3.1.2-bin-hadoop3.2")# SPARK_HOME
from pyspark.sql import SparkSession
import pyspark.sql.functions  as F
from pyspark.sql.functions import explode,split
from pyspark import SparkContext
 





In [10]:
sc=SparkContext()
session = SparkSession.builder.master("local[*]").getOrCreate()

In [11]:
df = session.read.csv("/content/imdb-dataset/title.akas.tsv", sep=r'\t', header=True).select("titleId","title")

In [12]:
df.show()

+---------+--------------------+
|  titleId|               title|
+---------+--------------------+
|tt0000001|Carmencita - span...|
|tt0000001|          Καρμενσίτα|
|tt0000001|          Карменсита|
|tt0000001|          Carmencita|
|tt0000001|          Carmencita|
|tt0000002|Le clown et ses c...|
|tt0000002|   A bohóc és kutyái|
|tt0000002|Le clown et ses c...|
|tt0000002|Clovnul si cainii...|
|tt0000002|  Клоун и его собаки|
|tt0000002|The Clown and His...|
|tt0000003|    Sarmanul Pierrot|
|tt0000003|     Szegény Pierrot|
|tt0000003|        Бедный Пьеро|
|tt0000003|      Pauvre Pierrot|
|tt0000003|        Poor Pierrot|
|tt0000003|      Pauvre Pierrot|
|tt0000004|         Un bon bock|
|tt0000004|      Un ţap de bere|
|tt0000004|         Un bon bock|
+---------+--------------------+
only showing top 20 rows



In [13]:
df.printSchema()

root
 |-- titleId: string (nullable = true)
 |-- title: string (nullable = true)



In [14]:
df2 = session.read.csv("/content/imdb-dataset/name.basics.tsv", sep=r'\t', header=True)

In [15]:
df2.show()

+---------+-------------------+---------+---------+--------------------+--------------------+
|   nconst|        primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+-------------------+---------+---------+--------------------+--------------------+
|nm0000001|       Fred Astaire|     1899|     1987|soundtrack,actor,...|tt0050419,tt00531...|
|nm0000002|      Lauren Bacall|     1924|     2014|  actress,soundtrack|tt0071877,tt01170...|
|nm0000003|    Brigitte Bardot|     1934|       \N|actress,soundtrac...|tt0054452,tt00491...|
|nm0000004|       John Belushi|     1949|     1982|actor,writer,soun...|tt0077975,tt00725...|
|nm0000005|     Ingmar Bergman|     1918|     2007|writer,director,a...|tt0069467,tt00509...|
|nm0000006|     Ingrid Bergman|     1915|     1982|actress,soundtrac...|tt0038109,tt00368...|
|nm0000007|    Humphrey Bogart|     1899|     1957|actor,soundtrack,...|tt0043265,tt00338...|
|nm0000008|      Marlon Brando|     1924|     2004|actor,sou

In [16]:
names= session.read.csv("/content/imdb-dataset/title.principals.tsv", sep=r'\t', header=True).sample(0.1)

In [17]:
names = names.where((names.category == "actor") | (names.category == "actress")).select("tconst","nconst")

In [18]:
names.show(truncate=False)

+---------+---------+
|tconst   |nconst   |
+---------+---------+
|tt0000018|nm3692071|
|tt0000026|nm2350007|
|tt0000026|nm0525907|
|tt0000061|nm4007970|
|tt0000112|nm0420198|
|tt0000121|nm0832461|
|tt0000165|nm0556371|
|tt0000215|nm0841389|
|tt0000235|nm0194945|
|tt0000246|nm0617588|
|tt0000247|nm0002504|
|tt0000305|nm0112631|
|tt0000310|nm0151209|
|tt0000376|nm0617272|
|tt0000439|nm0055661|
|tt0000442|nm0622273|
|tt0000452|nm0001908|
|tt0000480|nm0617588|
|tt0000487|nm0523316|
|tt0000488|nm0422465|
+---------+---------+
only showing top 20 rows



In [19]:
movie_names =df2.join(names,"nconst",'inner')

In [20]:
movie_names.show(truncate=False)

+---------+--------------+---------+---------+-------------------------+---------------------------------------+---------+
|nconst   |primaryName   |birthYear|deathYear|primaryProfession        |knownForTitles                         |tconst   |
+---------+--------------+---------+---------+-------------------------+---------------------------------------+---------+
|nm0000086|Louis de Funès|1914     |1983     |actor,writer,soundtrack  |tt0074103,tt0058135,tt0083109,tt0069747|tt0053076|
|nm0000086|Louis de Funès|1914     |1983     |actor,writer,soundtrack  |tt0074103,tt0058135,tt0083109,tt0069747|tt0054394|
|nm0000198|Gary Oldman   |1958     |\N       |actor,soundtrack,producer|tt0103874,tt1340800,tt0468569,tt4555426|tt0091954|
|nm0000198|Gary Oldman   |1958     |\N       |actor,soundtrack,producer|tt0103874,tt1340800,tt0468569,tt4555426|tt0095158|
|nm0000198|Gary Oldman   |1958     |\N       |actor,soundtrack,producer|tt0103874,tt1340800,tt0468569,tt4555426|tt0103874|
|nm0000198|Gary 

In [21]:
splited = movie_names.select("primaryName","knownForTitles",split("knownForTitles",",").alias("split"))
exploded = splited.select("primaryName",explode("split").alias("titleId"))
exploded.show()

+--------------+---------+
|   primaryName|  titleId|
+--------------+---------+
|Louis de Funès|tt0074103|
|Louis de Funès|tt0058135|
|Louis de Funès|tt0083109|
|Louis de Funès|tt0069747|
|Louis de Funès|tt0074103|
|Louis de Funès|tt0058135|
|Louis de Funès|tt0083109|
|Louis de Funès|tt0069747|
|   Gary Oldman|tt0103874|
|   Gary Oldman|tt1340800|
|   Gary Oldman|tt0468569|
|   Gary Oldman|tt4555426|
|   Gary Oldman|tt0103874|
|   Gary Oldman|tt1340800|
|   Gary Oldman|tt0468569|
|   Gary Oldman|tt4555426|
|   Gary Oldman|tt0103874|
|   Gary Oldman|tt1340800|
|   Gary Oldman|tt0468569|
|   Gary Oldman|tt4555426|
+--------------+---------+
only showing top 20 rows



In [22]:
exploded.where("titleId == 'tt0117057'").show()

+----------------+---------+
|     primaryName|  titleId|
+----------------+---------+
|Barbra Streisand|tt0117057|
|Barbra Streisand|tt0117057|
|    William Cain|tt0117057|
|  Lesley Garrett|tt0117057|
|      Jimmy Baio|tt0117057|
|      Jimmy Baio|tt0117057|
|      Jimmy Baio|tt0117057|
|      Jimmy Baio|tt0117057|
|      Jimmy Baio|tt0117057|
|      Jimmy Baio|tt0117057|
|      Jimmy Baio|tt0117057|
|      Jimmy Baio|tt0117057|
|      Jimmy Baio|tt0117057|
| Anne O'Sullivan|tt0117057|
|   Lauren Bacall|tt0117057|
|   Lauren Bacall|tt0117057|
|   Lauren Bacall|tt0117057|
|   Lauren Bacall|tt0117057|
|   Lauren Bacall|tt0117057|
|   Lauren Bacall|tt0117057|
+----------------+---------+
only showing top 20 rows



In [23]:
prepared_df = exploded.join(df,"titleId","inner")

In [24]:
 prepared_df.where("title == 'Lucille Love: The Girl of Mystery'").show(truncate=False)

+---------+-------------+---------------------------------+
|titleId  |primaryName  |title                            |
+---------+-------------+---------------------------------+
|tt0004272|Harry Schumm |Lucille Love: The Girl of Mystery|
|tt0004272|Harry Schumm |Lucille Love: The Girl of Mystery|
|tt0004272|Harry Schumm |Lucille Love: The Girl of Mystery|
|tt0004272|Harry Schumm |Lucille Love: The Girl of Mystery|
|tt0004272|Harry Schumm |Lucille Love: The Girl of Mystery|
|tt0004272|Harry Schumm |Lucille Love: The Girl of Mystery|
|tt0004272|Harry Schumm |Lucille Love: The Girl of Mystery|
|tt0004272|Harry Schumm |Lucille Love: The Girl of Mystery|
|tt0004272|Harry Schumm |Lucille Love: The Girl of Mystery|
|tt0004272|Harry Schumm |Lucille Love: The Girl of Mystery|
|tt0004272|Harry Schumm |Lucille Love: The Girl of Mystery|
|tt0004272|Harry Schumm |Lucille Love: The Girl of Mystery|
|tt0004272|Burton Law   |Lucille Love: The Girl of Mystery|
|tt0004272|Burton Law   |Lucille Love: T

In [25]:
prepared_df.groupBy("title","primaryName").count().show()

+--------------------+--------------------+-----+
|               title|         primaryName|count|
+--------------------+--------------------+-----+
|Den gaadefulde Villa|           Hal Forde|    1|
|     Paradise Garden|         George Hupp|    2|
|La conquête de Ja...|Eunice Murdock Moore|    2|
|Dürfen wir schwei...|        Bella Polini|    2|
|Torchy Blane in P...|           Lola Lane|   10|
|      Sarlo Diktatör|          Eddie Dunn|    7|
|           O Ditador|    Paulette Goddard|    3|
|     Veliki diktator|           Nita Pike|    3|
|  O Grande Escândalo|          Frank Orth|   14|
| Его девушка Пятница|          Helen Mack|    2|
|Dziewczyna Pietaszek|        Ernest Truex|   25|
|     Siste sensasjon|        Regis Toomey|   32|
|La croix de Lorraine|    Jack Edwards Jr.|    2|
|   Éljen a szerelem!|        Ray McDonald|    1|
|       Leve kärleken|        Ray McDonald|    1|
|I doukissa kai o ...|       Árpád Lehotay|    1|
|      Saiturin joulu|     Ernest Thesiger|    3|


In [26]:
basket  =prepared_df.groupBy("title").agg(F.collect_set("primaryName")).cache()

In [27]:
from pyspark.ml.fpm import FPGrowth
## modeling FP Growth alforithm to get Frequent Itemsets

fpGrowth = FPGrowth(itemsCol="collect_set(primaryName)", minSupport=0.0001, minConfidence=0.001)
model = fpGrowth.fit(basket)

In [28]:
model.freqItemsets.show(truncate= False)

+-------------------+----+
|items              |freq|
+-------------------+----+
|[Steven Spielberg] |144 |
|[Paul Walker]      |131 |
|[Marina Sirtis]    |126 |
|[Paul Herman]      |109 |
|[Nick Keller]      |108 |
|[Charles Cyphers]  |122 |
|[Nelly]            |119 |
|[Peter Bull]       |116 |
|[Richard Cox]      |114 |
|[Melanie Hill]     |112 |
|[Robby Robinson]   |110 |
|[David Bradley]    |271 |
|[Pat O'Brien]      |122 |
|[Byron Thames]     |119 |
|[Robert Pastorelli]|116 |
|[David Goldsmith]  |114 |
|[Peter Ramsey]     |112 |
|[Matthew Walker]   |242 |
|[Michael Anthony]  |143 |
|[Edward Binns]     |131 |
+-------------------+----+
only showing top 20 rows



In [29]:
model.associationRules.show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------+---------------------+----------+------------------+---------------------+
|antecedent                                                                                                             |consequent           |confidence|lift              |support              |
+-----------------------------------------------------------------------------------------------------------------------+---------------------+----------+------------------+---------------------+
|[Stephen Hunter, William Kircher, Sylvester McCoy, Dean O'Gorman, Graham McTavish, Ken Stott, Aidan Turner, Adam Brown]|[Peter Hambleton]    |1.0       |8218.576923076922 |1.1231590720834133E-4|
|[Stephen Hunter, William Kircher, Sylvester McCoy, Dean O'Gorman, Graham McTavish, Ken Stott, Aidan Turner, Adam Brown]|[Richard Armitage]   |1.0       |7368.379310344827 |1.1231590720834133E-4|
|[Stephen Hunter, Wi

In [30]:
lists = basket.select("collect_set(primaryName)").withColumnRenamed("collect_set(primaryName)","primary_names").sample(0.00001).collect()
sno_id_array = [ row.primary_names for row in lists]
sno_id_array

[['Christos Doxaras'],
 ['Leon Errol', 'Gladys Hurlbut', 'Paul Hartman'],
 ['Tony Martin', 'Hazel Shaw', 'Charles Dale', 'Joe Smith'],
 ['Dorléac'],
 ['Gilbert Green', 'Ross Martin'],
 ['Rika Dialyna', 'Alexander Welbat'],
 ['CJ Jones',
  'Billy Chase Goforth',
  'Victor Pongonis',
  'Lauren Harper',
  'Mark Landon Smith',
  'Mitchell Crisp'],
 ['Greg Strempler'],
 ['Stephen Rea',
  'Catlin Adams',
  'India Eisley',
  'Cameron Grierson',
  'Jacob Blair',
  'Kris Holden-Ried',
  'Nils Lagergren',
  'Robert Lawrenson'],
 ['Candy Jones'],
 ['John Connon',
  'Joan Gregson',
  'Jay Ryan',
  'Bill Skarsgård',
  'Ryan Kiera Armstrong',
  'Jake Weary',
  'Jack Dylan Grazer',
  'Liam MacDonald',
  'Sebastian Nagy',
  'Billy Merasty',
  'Angelica Alejandro',
  'Rob Ramsay',
  'Isaiah Mustafa',
  'Chris Tarpos',
  'Brandon Crane',
  'Jeremy Ray Taylor',
  'Molly Atkinson',
  'Will Beinbrink',
  'Robert C. Eaton',
  'Janet Porter',
  'Owen Teague',
  'Divan Meyer',
  'Jake Sim',
  'Logan Thompson'

In [31]:
lblitems =sc.parallelize(sno_id_array)
wlitems=lblitems.flatMap(lambda line:line)

In [32]:
uniqueItems = wlitems.distinct()
supportRdd = wlitems.map(lambda item: (item , 1))

def sumOparator(x,y):
    return x+y

supportRdd = supportRdd.reduceByKey(sumOparator) 
supports = supportRdd.map(lambda item: item[1]) # Return only support values

In [33]:

minSupport = supports.min()

minSupport = 2 if minSupport == 1 else minSupport

supportRdd = supportRdd.filter(lambda item: item[1] >= minSupport )

baseRdd = supportRdd.map(lambda item: ([item[0]] , item[1])) 
print('1 . Table has crated...') 

supportRdd = supportRdd.map(lambda item: item[0])
supportRddCart = supportRdd

1 . Table has crated...


In [34]:

def removeReplica(record):

    if(isinstance(record[0], tuple)):
        x1 = record[0]
        x2 = record[1]
    else:
        x1 = [record[0]]
        x2 = record[1]

    if(any(x == x2 for x in x1) == False):
        a = list(x1)
        a.append(x2)
        a.sort()
        result = tuple(a)
        return result 
    else:
        return x1

In [35]:
c = 2 

while(supportRdd.isEmpty() == False):

    combined = supportRdd.cartesian(uniqueItems)
    combined = combined.map(lambda item: removeReplica(item))
  
    combined = combined.filter(lambda item: len(item) == c)
    combined = combined.distinct()

    
    combined_2 = combined.cartesian(lblitems)
    combined_2 = combined_2.filter(lambda item: all(x in item[1] for x in item[0]))
    
    combined_2 = combined_2.map(lambda item: item[0])
    combined_2 = combined_2.map(lambda item: (item , 1))
    combined_2 = combined_2.reduceByKey(sumOparator)
    combined_2 = combined_2.filter(lambda item: item[1] >= minSupport)

    baseRdd = baseRdd.union(combined_2)
    
    combined_2 = combined_2.map(lambda item: item[0])
    supportRdd = combined_2
    print(c ,'. Table has crated... ')
    c = c+1 

In [36]:
class Filter():

    def __init__(self):
        
        self.stages = 1


    def filterForConf(self, item , total):
        
        if(len(item[0][0]) > len(item[1][0])  ):
            if(self.checkItemSets(item[0][0] , item[1][0]) == False):
                pass
            else:
                return (item)       
        else:
            pass  
        self.stages = self.stages + 1

    def checkItemSets(self, item_1 , item_2):

        if(len(item_1) > len(item_2)):
            return all(any(k == l for k in item_1 ) for l in item_2)
        else:
            return all(any(k == l for k in item_2 ) for l in item_1)


    def calculateConfidence(self, item):

        parent = set(item[0][0])
        
        if(isinstance(item[1][0] , str)):
            child  = set([item[1][0]])
        else:
            child  = set(item[1][0])
        
        parentSupport = item[0][1]
        childSupport = item[1][1]
       

        support = (parentSupport / childSupport)*100

        return list([ list(child) ,  list(parent.difference(child)) , support ])

In [37]:
calcuItems = baseRdd.cartesian(baseRdd)

In [38]:
ff = Filter()

In [39]:
total = calcuItems.count()

In [40]:
print('# : Aggregated support values preparing for the confidence calculatations')
baseRddConfidence = calcuItems.filter(lambda item: ff.filterForConf(item , total))

# : Aggregated support values preparing for the confidence calculatations


In [41]:
print('# : Aggregated support values are ready !')
baseRddConfidence = baseRddConfidence.map(lambda item: ff.calculateConfidence(item))

# : Aggregated support values are ready !


In [42]:
print(baseRddConfidence.collect())
## No confidence could be made in current use case 

[]
